# AlphaSnake 10x10 - Paper-Faithful para Vast.ai

Objetivo:
- Entrenar un agente Snake con enfoque AlphaZero (MCTS + Policy/Value ResNet) replicando la receta 10x10 del paper.
- Exportar un modelo ONNX compatible con `/Users/JuanCamiloTorresUrrego/Documents/tiktok/arcade/games/snake/ai.html`.

Criterios de exito:
1. Perfil `paper_strict` (default) con hiperparametros paper-faithful.
2. Perfil `smoke` para validar pipeline end-to-end en pocos minutos.
3. ONNX con contrato fijo: input `state`, outputs `policy` y `value`.


In [ ]:
# ============================================================
# Cell 1: Setup reproducible + rutas de artefactos
# ============================================================
import os
import random
import numpy as np
import torch

SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(SEED)

cpu_threads = max(1, min(4, int(os.environ.get('ALPHASNAKE_CPU_THREADS', '4'))))
torch.set_num_threads(cpu_threads)
torch.set_num_interop_threads(1)

DEFAULT_VAST_SAVE_DIR = '/workspace/alphasnake_paper_10x10'
if os.path.exists('/workspace'):
    SAVE_DIR = os.environ.get('ALPHASNAKE_SAVE_DIR', DEFAULT_VAST_SAVE_DIR)
else:
    SAVE_DIR = os.environ.get('ALPHASNAKE_SAVE_DIR', os.path.join(os.getcwd(), 'alphasnake_paper_10x10'))

os.makedirs(SAVE_DIR, exist_ok=True)

print(f'PyTorch: {torch.__version__}')
print(f'CUDA disponible: {torch.cuda.is_available()}')
print(f'CPU threads torch: {torch.get_num_threads()}')
print(f'Directorio de checkpoints/modelo: {SAVE_DIR}')


## Perfil de ejecucion

Este notebook define dos perfiles:
- `paper_strict` (default): reproduccion fiel de parametros paper 10x10.
- `smoke`: validacion rapida del pipeline (self-play -> train -> eval -> export).


In [ ]:
# Perfil activo (puedes cambiarlo via variable de entorno ALPHASNAKE_PROFILE)
PROFILE = os.environ.get('ALPHASNAKE_PROFILE', 'paper_strict').strip().lower()
ALLOWED_PROFILES = {'paper_strict', 'smoke'}
if PROFILE not in ALLOWED_PROFILES:
    raise ValueError(f'Perfil invalido: {PROFILE}. Opciones: {sorted(ALLOWED_PROFILES)}')

print(f'Perfil seleccionado: {PROFILE}')


In [ ]:
# ============================================================
# Cell 2: Imports + Config central (TrainConfig)
# ============================================================
import dataclasses
from dataclasses import dataclass
from collections import deque
from queue import Empty
import concurrent.futures
import math
import multiprocessing
import os
import pickle
import random
import threading
import time

import numpy as np
from tqdm.auto import tqdm

import torch
import torch.nn as nn
import torch.nn.functional as F


@dataclass
class TrainConfig:
    board_size: int = 10
    max_steps: int = 1000

    net_channels: int = 64
    net_blocks: int = 6

    num_simulations: int = 400
    c_puct: float = 1.0
    dir_alpha: float = 0.03
    dir_eps: float = 0.25
    temp_init: float = 1.0
    temp_final: float = 0.0
    temp_decay_move: int = 30
    food_samples: int = 8

    lr: float = 1e-3
    weight_decay: float = 1e-4
    batch_size: int = 128
    buffer_size: int = 200_000
    epochs_per_iter: int = 10

    games_per_iter: int = 1000
    max_iterations: int = 200

    eval_games: int = 200
    accept_threshold: float = 0.55

    save_dir: str = SAVE_DIR
    checkpoint_interval: int = 1

    selfplay_workers: int = 10
    inference_batch_size: int = 128
    worker_oversub_factor: float = 3.0
    inference_timeout_ms: int = 20
    predict_timeout_s: int = 180
    use_amp: bool = True

    verbose_game_updates: bool = False
    game_log_interval: int = 10
    game_tick_moves: int = 10
    heartbeat_seconds: int = 20


def build_config(profile: str, save_dir: str) -> TrainConfig:
    if profile == 'smoke':
        return TrainConfig(
            save_dir=save_dir,
            num_simulations=80,
            games_per_iter=24,
            max_iterations=1,
            eval_games=20,
            epochs_per_iter=2,
            batch_size=64,
            buffer_size=20_000,
            food_samples=4,
            selfplay_workers=4,
            inference_batch_size=64,
            inference_timeout_ms=10,
            use_amp=True,
        )
    return TrainConfig(save_dir=save_dir)


config = build_config(PROFILE, SAVE_DIR)
Config = TrainConfig

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
if torch.cuda.is_available():
    torch.backends.cudnn.benchmark = True
    torch.backends.cuda.matmul.allow_tf32 = True
    torch.backends.cudnn.allow_tf32 = True
    torch.set_float32_matmul_precision('high')

print(f'Device: {device}')
print(f'Config activa: {dataclasses.asdict(config)}')


### Takeaway - Config
- `paper_strict` mantiene la receta paper 10x10 como default.
- `smoke` permite validar el pipeline end-to-end sin esperar horas.
- El directorio objetivo de Vast.ai queda en `/workspace/alphasnake_paper_10x10`.


## Entorno Snake (paper-faithful)

- Grid: `10x10`
- Acciones: `0=UP, 1=DOWN, 2=LEFT, 3=RIGHT`
- Sin reversa directa
- Observacion: `(4, 10, 10)` en channels-first
- Recompensa exacta: `+1` comida, `-1` muerte, `0` otro


In [ ]:
# ============================================================
# Cell 3: Entorno Snake
# ============================================================

class SnakeEnv:
    """
    Snake environment estilo AlphaSnake.
    Grid board_size x board_size, 4 acciones, observacion (4, H, W).
    """
    # 0=UP, 1=DOWN, 2=LEFT, 3=RIGHT
    ACTIONS = {0: (0, -1), 1: (0, 1), 2: (-1, 0), 3: (1, 0)}
    OPPOSITES = {0: 1, 1: 0, 2: 3, 3: 2}
    ACTION_NAMES = {0: 'UP', 1: 'DOWN', 2: 'LEFT', 3: 'RIGHT'}

    def __init__(self, board_size=10, max_steps=1000):
        self.board_size = board_size
        self.max_steps = max_steps
        self.max_score = board_size * board_size - 3  # 97 para 10x10
        self.reset()

    def reset(self):
        mid = self.board_size // 2
        # Snake empieza en el centro, apuntando a la derecha
        self.snake = deque([(mid, mid), (mid - 1, mid), (mid - 2, mid)])
        self.direction = 3  # RIGHT
        self.food = None
        self.done = False
        self.steps = 0
        self.score = 0
        self._place_food()
        return self.get_state()

    def _get_free_cells(self):
        snake_set = set(self.snake)
        return [(x, y) for y in range(self.board_size)
                for x in range(self.board_size)
                if (x, y) not in snake_set]

    def _place_food(self):
        free = self._get_free_cells()
        if free:
            self.food = random.choice(free)
        else:
            # Tablero lleno = victoria
            self.food = None
            self.done = True

    def get_state(self):
        """Retorna observacion (4, board_size, board_size) float32."""
        s = np.zeros((4, self.board_size, self.board_size), dtype=np.float32)
        # Canal 0: cuerpo
        for x, y in self.snake:
            s[0, y, x] = 1.0
        # Canal 1: cabeza
        hx, hy = self.snake[0]
        s[1, hy, hx] = 1.0
        # Canal 2: comida
        if self.food:
            fx, fy = self.food
            s[2, fy, fx] = 1.0
        # Canal 3: direccion (valor constante normalizado)
        # UP=0.25, DOWN=0.5, LEFT=0.75, RIGHT=1.0
        s[3, :, :] = (self.direction + 1) / 4.0
        return s

    def valid_actions(self):
        """Acciones validas (excluye reversa directa)."""
        rev = self.OPPOSITES[self.direction]
        return [a for a in range(4) if a != rev]

    def step(self, action):
        """
        Ejecuta accion. Retorna (reward, done).
        reward: +1.0 comida, -1.0 muerte, 0.0 otro.
        """
        if self.done:
            return 0.0, True

        # Prohibir reversa directa: si intenta reversa, mantener direccion
        if action == self.OPPOSITES[self.direction]:
            action = self.direction

        self.direction = action
        dx, dy = self.ACTIONS[action]
        hx, hy = self.snake[0]
        nx, ny = hx + dx, hy + dy

        # Colision con pared
        if nx < 0 or nx >= self.board_size or ny < 0 or ny >= self.board_size:
            self.done = True
            return -1.0, True

        # Verificar si comera comida (antes de mover)
        ate_food = self.food is not None and (nx, ny) == self.food

        # Colision con cuerpo
        # Si NO come comida, la cola se movera, asi que es valido ir a la pos actual de la cola
        body_set = set(self.snake)
        if not ate_food:
            body_set.discard(self.snake[-1])
        if (nx, ny) in body_set:
            self.done = True
            return -1.0, True

        # Mover
        self.snake.appendleft((nx, ny))
        if ate_food:
            self.score += 1
            self._place_food()
            if self.done:  # _place_food puso done=True si el tablero esta lleno
                return 1.0, True
        else:
            self.snake.pop()

        self.steps += 1
        if self.steps >= self.max_steps:
            self.done = True
            return 0.0, True

        return (1.0 if ate_food else 0.0), False

    def is_win(self):
        """True si la serpiente lleno todo el tablero."""
        return len(self.snake) >= self.board_size ** 2

    def clone(self):
        """Copia eficiente del entorno."""
        env = SnakeEnv.__new__(SnakeEnv)
        env.board_size = self.board_size
        env.max_steps = self.max_steps
        env.max_score = self.max_score
        env.snake = deque(self.snake)
        env.direction = self.direction
        env.food = self.food
        env.done = self.done
        env.steps = self.steps
        env.score = self.score
        return env

# --- Test rapido ---
env = SnakeEnv(10, 1000)
s = env.reset()
print(f"State shape: {s.shape}")
print(f"Snake length: {len(env.snake)}, food: {env.food}")
print(f"Valid actions: {[env.ACTION_NAMES[a] for a in env.valid_actions()]}")

# Jugar unos pasos random
for _ in range(20):
    a = random.choice(env.valid_actions())
    r, done = env.step(a)
    if done:
        print(f"Game over! Score: {env.score}, Win: {env.is_win()}")
        break
else:
    print(f"After 20 steps: score={env.score}, length={len(env.snake)}")

In [ ]:
# ============================================================
# Cell 3b: Sanity checks del entorno (contrato minimo)
# ============================================================

def run_env_sanity_checks():
    env = SnakeEnv(10, 1000)
    s = env.reset()
    assert s.shape == (4, 10, 10), f'Shape invalido: {s.shape}'

    env = SnakeEnv(10, 1000)
    env.reset()
    env.direction = 3
    _, _ = env.step(2)
    assert env.direction == 3, 'Se permitio reversa directa'

    env = SnakeEnv(10, 1000)
    env.reset()
    hx, hy = env.snake[0]
    env.direction = 3
    env.food = (hx + 1, hy)
    r, done = env.step(3)
    assert r == 1.0 and not done, f'Reward comer invalido: r={r}, done={done}'

    env = SnakeEnv(10, 1000)
    env.reset()
    env.food = (0, 0)
    r, done = env.step(3)
    assert r == 0.0 and not done, f'Reward paso invalido: r={r}, done={done}'

    env = SnakeEnv(10, 1000)
    env.reset()
    env.snake = deque([(0, 0), (0, 1), (0, 2)])
    env.direction = 2
    r, done = env.step(2)
    assert r == -1.0 and done, f'Reward muerte invalido: r={r}, done={done}'

run_env_sanity_checks()
print('Sanity checks del entorno: OK')


### Takeaway - Entorno
El contrato del entorno quedo alineado con paper: acciones discretas, no reversa,
observacion `(4,10,10)` y rewards exactos `+1/0/-1`.


## Red Neuronal: AlphaSnakeNet

Arquitectura AlphaZero reducida:
- Stem conv `3x3` con 64 canales
- `6` bloques residuales
- Policy head -> `softmax` de 4 acciones
- Value head -> `tanh` en `[-1, 1]`


In [ ]:
# ============================================================
# Cell 4: Red Neuronal — AlphaSnakeNet
# ============================================================

class ResBlock(nn.Module):
    """Bloque residual: Conv→BN→ReLU→Conv→BN→skip→ReLU"""
    def __init__(self, channels):
        super().__init__()
        self.conv1 = nn.Conv2d(channels, channels, 3, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(channels)
        self.conv2 = nn.Conv2d(channels, channels, 3, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(channels)

    def forward(self, x):
        residual = x
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out = F.relu(out + residual)
        return out


class AlphaSnakeNet(nn.Module):
    """
    Policy + Value network estilo AlphaZero.
    Input:  (batch, 4, board_size, board_size)
    Output: policy (batch, 4), value (batch, 1)
    """
    def __init__(self, in_channels=4, num_blocks=6, channels=64, board_size=10):
        super().__init__()
        self.board_size = board_size

        # Stem
        self.stem = nn.Sequential(
            nn.Conv2d(in_channels, channels, 3, padding=1, bias=False),
            nn.BatchNorm2d(channels),
            nn.ReLU()
        )

        # Residual tower
        self.res_tower = nn.Sequential(*[ResBlock(channels) for _ in range(num_blocks)])

        # Policy head
        self.policy_conv = nn.Conv2d(channels, 2, 1, bias=False)
        self.policy_bn = nn.BatchNorm2d(2)
        self.policy_fc = nn.Linear(2 * board_size * board_size, 4)

        # Value head
        self.value_conv = nn.Conv2d(channels, 1, 1, bias=False)
        self.value_bn = nn.BatchNorm2d(1)
        self.value_fc1 = nn.Linear(board_size * board_size, 64)
        self.value_fc2 = nn.Linear(64, 1)

    def forward(self, x):
        # Shared trunk
        x = self.stem(x)
        x = self.res_tower(x)

        # Policy
        p = F.relu(self.policy_bn(self.policy_conv(x)))
        p = p.view(p.size(0), -1)
        p = F.softmax(self.policy_fc(p), dim=1)

        # Value
        v = F.relu(self.value_bn(self.value_conv(x)))
        v = v.view(v.size(0), -1)
        v = F.relu(self.value_fc1(v))
        v = torch.tanh(self.value_fc2(v))

        return p, v

# --- Test ---
net = AlphaSnakeNet(
    in_channels=4,
    num_blocks=config.net_blocks,
    channels=config.net_channels,
    board_size=config.board_size
).to(device)

total_params = sum(p.numel() for p in net.parameters())
print(f"AlphaSnakeNet: {total_params:,} parametros")

dummy = torch.randn(1, 4, 10, 10).to(device)
pol, val = net(dummy)
print(f"Policy shape: {pol.shape}, sum={pol.sum().item():.4f}")
print(f"Value shape: {val.shape}, val={val.item():.4f}")
del net, dummy  # liberar memoria

## MCTS - Monte Carlo Tree Search

Implementacion AlphaZero con:
- UCB PUCT
- Dirichlet noise en raiz (`alpha=0.03`, `eps=0.25`)
- Mascara de acciones invalidas
- Manejo de estocasticidad de comida (`food_samples`)


In [ ]:
# ============================================================
# Cell 5: MCTS — Monte Carlo Tree Search
# ============================================================

class MCTSNode:
    """Nodo del arbol MCTS."""
    __slots__ = ['prior', 'visit_count', 'value_sum', 'children',
                 'is_expanded', 'env', 'food_eaten']

    def __init__(self, prior=0.0):
        self.prior = prior
        self.visit_count = 0
        self.value_sum = 0.0
        self.children = {}        # action (int) -> MCTSNode
        self.is_expanded = False
        self.env = None           # SnakeEnv snapshot
        self.food_eaten = False   # True si la transicion a este nodo comio comida

    def value(self):
        if self.visit_count == 0:
            return 0.0
        return self.value_sum / self.visit_count


class MCTS:
    """
    Monte Carlo Tree Search estilo AlphaZero con soporte
    para estocasticidad de comida (food sampling).
    Acepta predict_fn y predict_batch_fn para permitir
    batching de inferencia desde multiples hilos.
    """
    def __init__(self, predict_fn, cfg, predict_batch_fn=None):
        self.predict_fn = predict_fn
        self.predict_batch_fn = predict_batch_fn or (lambda states: [predict_fn(s) for s in states])
        self.c_puct = cfg.c_puct
        self.num_simulations = cfg.num_simulations
        self.dir_alpha = cfg.dir_alpha
        self.dir_eps = cfg.dir_eps
        self.food_samples = cfg.food_samples

    def _predict(self, state_np):
        """Inferir policy y value."""
        return self.predict_fn(state_np)

    def _ucb_score(self, parent, child):
        """UCB score clasico AlphaZero."""
        q = child.value()
        u = self.c_puct * child.prior * math.sqrt(parent.visit_count) / (1 + child.visit_count)
        return q + u

    def _select_child(self, node):
        """Seleccionar hijo con mayor UCB score."""
        best_score = -float('inf')
        best_action = -1
        best_child = None
        for action, child in node.children.items():
            score = self._ucb_score(node, child)
            if score > best_score:
                best_score = score
                best_action = action
                best_child = child
        return best_action, best_child

    def _expand(self, node):
        """
        Expandir nodo hoja: evaluar con la red, crear hijos.
        Maneja estocasticidad de comida promediando valores.
        Retorna el valor del nodo.
        """
        env = node.env
        if env.done:
            return 1.0 if env.is_win() else -1.0

        policy, value = self._predict(env.get_state())

        # --- Food stochasticity (batched via predict_batch_fn) ---
        if node.food_eaten and self.food_samples > 1:
            snake_set = set(env.snake)
            free = [(x, y) for y in range(env.board_size)
                    for x in range(env.board_size)
                    if (x, y) not in snake_set and (x, y) != env.food]
            k = min(self.food_samples - 1, len(free))
            if k > 0:
                sampled_positions = random.sample(free, k)
                food_states = []
                for food_pos in sampled_positions:
                    env_copy = env.clone()
                    env_copy.food = food_pos
                    food_states.append(env_copy.get_state())
                results = self.predict_batch_fn(food_states)
                food_values = [r[1] for r in results]
                value = (value + sum(food_values)) / (1 + len(food_values))

        # Mascara de acciones validas y renormalizacion
        valid = env.valid_actions()
        mask = np.zeros(4, dtype=np.float32)
        for a in valid:
            mask[a] = 1.0
        policy = policy * mask
        total = policy.sum()
        if total > 0:
            policy /= total
        else:
            policy = mask / mask.sum()

        # Crear hijos
        for a in valid:
            node.children[a] = MCTSNode(prior=policy[a])

        node.is_expanded = True
        return value

    def _add_dirichlet_noise(self, node):
        """Agregar Dirichlet noise al root para exploracion."""
        actions = list(node.children.keys())
        if not actions:
            return
        noise = np.random.dirichlet([self.dir_alpha] * len(actions))
        for i, a in enumerate(actions):
            node.children[a].prior = (
                (1 - self.dir_eps) * node.children[a].prior +
                self.dir_eps * noise[i]
            )

    def search(self, root_env, temperature=1.0):
        """
        Ejecutar MCTS completo desde un estado raiz.
        Retorna distribucion de probabilidad sobre acciones (4,).
        """
        root = MCTSNode()
        root.env = root_env.clone()

        # Expandir raiz
        self._expand(root)
        self._add_dirichlet_noise(root)

        # Simulaciones
        for _ in range(self.num_simulations):
            node = root
            path = [node]

            # --- SELECT ---
            while node.is_expanded and node.children:
                action, child = self._select_child(node)

                # Computacion lazy del estado del hijo
                if child.env is None:
                    env_copy = node.env.clone()
                    old_score = env_copy.score
                    env_copy.step(action)
                    child.env = env_copy
                    child.food_eaten = (env_copy.score > old_score and not env_copy.done)

                node = child
                path.append(node)

            # --- EXPAND & EVALUATE ---
            if not node.is_expanded:
                value = self._expand(node)
            else:
                # Nodo terminal (expanded pero sin hijos)
                value = 1.0 if node.env.is_win() else -1.0

            # --- BACKUP ---
            for n in reversed(path):
                n.visit_count += 1
                n.value_sum += value

        # --- Construir distribucion de acciones ---
        visits = np.zeros(4, dtype=np.float32)
        for a, child in root.children.items():
            visits[a] = child.visit_count

        if temperature == 0:
            # Greedy
            best = np.argmax(visits)
            probs = np.zeros(4, dtype=np.float32)
            probs[best] = 1.0
        else:
            # Con temperatura
            visits_temp = np.power(visits, 1.0 / temperature)
            total = visits_temp.sum()
            if total > 0:
                probs = visits_temp / total
            else:
                probs = np.ones(4, dtype=np.float32) / 4.0

        return probs

print("MCTS implementado correctamente.")

### Takeaway - MCTS
La busqueda incluye ruido Dirichlet en raiz para exploracion durante self-play y
muestreo de comida (`food_samples`) para manejar estocasticidad del entorno.


In [ ]:
# ============================================================
# Cell 5b: InferenceBatcher — Batching de GPU para self-play paralelo
# ============================================================
import queue as queue_module
import concurrent.futures

class InferenceBatcher:
    """
    Recolecta requests de inferencia de multiples hilos y los
    procesa en batches en la GPU. Esto maximiza la utilizacion
    de la GPU durante self-play paralelo.
    """
    def __init__(self, net, device, max_batch=64, timeout_ms=3):
        self.net = net
        self.device = device
        self.max_batch = max_batch
        self.timeout = timeout_ms / 1000.0
        self._queue = queue_module.Queue()
        self._running = False
        self._thread = None
        self._total_calls = 0
        self._total_batches = 0

    def start(self):
        self._running = True
        self._total_calls = 0
        self._total_batches = 0
        self._thread = threading.Thread(target=self._worker, daemon=True)
        self._thread.start()
        return self

    def stop(self):
        self._running = False
        self._queue.put(None)  # sentinel
        if self._thread:
            self._thread.join(timeout=10)

    def submit(self, state_np):
        """Non-blocking: enviar estado, retorna Future."""
        f = concurrent.futures.Future()
        self._queue.put((state_np, f))
        return f

    def predict(self, state_np):
        """Blocking: enviar estado, esperar resultado (policy, value)."""
        return self.submit(state_np).result()

    def predict_batch(self, states_list):
        """Enviar multiples estados, esperar todos los resultados."""
        if not states_list:
            return []
        futures = [self.submit(s) for s in states_list]
        return [f.result() for f in futures]

    def stats(self):
        avg = self._total_calls / max(self._total_batches, 1)
        return self._total_calls, self._total_batches, avg

    def _worker(self):
        while self._running:
            batch = []
            # Esperar primer request
            try:
                item = self._queue.get(timeout=1.0)
            except queue_module.Empty:
                continue
            if item is None:
                break
            batch.append(item)

            # Recolectar mas requests (hasta max_batch o timeout)
            deadline = time.time() + self.timeout
            while len(batch) < self.max_batch:
                remaining = deadline - time.time()
                if remaining <= 0:
                    break
                try:
                    item = self._queue.get(timeout=max(0.0001, remaining))
                except queue_module.Empty:
                    break
                if item is None:
                    self._running = False
                    break
                batch.append(item)

            if not batch:
                continue

            states = [b[0] for b in batch]
            futures_list = [b[1] for b in batch]
            self._total_calls += len(batch)
            self._total_batches += 1

            try:
                arr = np.stack(states)
                t = torch.as_tensor(arr, device=self.device)
                with torch.no_grad():
                    policies, values = self.net(t)
                p_np = policies.cpu().numpy()
                v_np = values.cpu().numpy()
                for i, f in enumerate(futures_list):
                    f.set_result((p_np[i], float(v_np[i, 0])))
            except Exception as e:
                for f in futures_list:
                    if not f.done():
                        f.set_exception(e)


def make_predict_fn(net, device):
    """Crear funcion de prediccion directa (para uso single-thread, eval)."""
    @torch.no_grad()
    def predict(state_np):
        t = torch.as_tensor(state_np, device=device).unsqueeze(0)
        p, v = net(t)
        return p[0].cpu().numpy(), v.item()
    return predict

print("InferenceBatcher implementado.")

## Self-play -> Replay Buffer -> Train -> Eval

Pipeline por iteracion:
1. Self-play con MCTS para generar `(state, pi, z)`
2. Entrenamiento por batches en replay buffer
3. Evaluacion MCTS real
4. Champion update + checkpoint


In [ ]:
# ============================================================
# Cell 6: Self-Play
# ============================================================

def self_play_game(predict_fn, cfg, predict_batch_fn=None, progress_cb=None):
    """
    Jugar una partida completa usando MCTS.
    predict_fn: callable(state_np) -> (policy_np, value_float)
    predict_batch_fn: callable(list[state_np]) -> list[(policy, value)]
    progress_cb: callable(move_count, score, length) opcional para telemetria.
    Retorna: (examples, is_win, score, num_moves)
    """
    env = SnakeEnv(cfg.board_size, cfg.max_steps)
    mcts = MCTS(predict_fn, cfg, predict_batch_fn)
    env.reset()

    states = []
    policies = []
    move_count = 0

    while not env.done:
        # Temperatura: 1.0 hasta move temp_decay_move, luego 0.0
        temp = cfg.temp_init if move_count < cfg.temp_decay_move else cfg.temp_final

        # Busqueda MCTS
        pi = mcts.search(env, temperature=temp)

        # Guardar datos de entrenamiento
        states.append(env.get_state().copy())
        policies.append(pi.copy())

        # Seleccionar accion
        if temp == 0:
            action = int(np.argmax(pi))
        else:
            action = int(np.random.choice(4, p=pi))

        # Ejecutar
        env.step(action)
        move_count += 1
        if progress_cb is not None:
            tick_every = max(1, int(getattr(cfg, "game_tick_moves", 10)))
            if move_count == 1 or move_count % tick_every == 0:
                progress_cb(move_count, env.score, len(env.snake))

    # Determinar resultado: +1 si gano (lleno tablero), -1 si no
    z = 1.0 if env.is_win() else -1.0

    # Crear ejemplos de entrenamiento
    examples = [(s, p, z) for s, p in zip(states, policies)]
    return examples, env.is_win(), env.score, move_count, len(env.snake)


def run_n_games_with_predict(n, predict_fn, predict_batch_fn, progress_queue, worker_id, cfg):
    """
    Ejecutar n partidas de self-play (thread-safe), usando funciones de inferencia ya construidas.
    """
    all_examples = []
    wins = 0
    scores_list = []
    moves_list = []
    lengths_list = []

    if progress_queue is not None:
        progress_queue.put({"type": "worker_start", "worker": worker_id, "games": n})

    for game_idx in range(n):
        if progress_queue is not None:
            progress_queue.put({"type": "game_start", "worker": worker_id, "game_local": game_idx + 1})

        def _progress_cb(move_count, score_now, len_now):
            if progress_queue is not None:
                progress_queue.put({
                    "type": "game_tick",
                    "worker": worker_id,
                    "game_local": game_idx + 1,
                    "moves": move_count,
                    "score": score_now,
                    "length": len_now,
                })

        try:
            examples, won, score, moves, length = self_play_game(
                predict_fn,
                cfg,
                predict_batch_fn,
                progress_cb=_progress_cb,
            )
        except Exception as exc:
            if progress_queue is not None:
                progress_queue.put({
                    "type": "worker_error",
                    "worker": worker_id,
                    "game_local": game_idx + 1,
                    "message": f"{type(exc).__name__}: {exc}",
                })
            raise

        all_examples.extend(examples)
        if won:
            wins += 1
        scores_list.append(score)
        moves_list.append(moves)
        lengths_list.append(length)

        if progress_queue is not None:
            progress_queue.put({
                "type": "game_end",
                "worker": worker_id,
                "game_local": game_idx + 1,
                "won": won,
                "score": score,
                "moves": moves,
                "length": length,
                "examples": len(examples),
            })

    return all_examples, wins, scores_list, moves_list, lengths_list


def run_n_games(n, request_queue, response_queue, progress_queue, worker_id, cfg):
    """
    Compat con version de procesos: arma predict_fn sobre colas y delega al runner generico.
    """
    def predict(state_np):
        request_queue.put((worker_id, False, state_np))
        timeout_s = float(getattr(cfg, "predict_timeout_s", 180))
        try:
            resp = response_queue.get(timeout=timeout_s)
        except Empty:
            if progress_queue is not None:
                progress_queue.put({
                    "type": "worker_error",
                    "worker": worker_id,
                    "game_local": -1,
                    "message": f"Timeout esperando inferencia ({timeout_s:.0f}s)",
                })
            raise TimeoutError(f"Worker {worker_id}: timeout esperando inferencia ({timeout_s:.0f}s)")

        if isinstance(resp, dict) and resp.get("type") == "inference_error":
            msg = resp.get("message", "error desconocido de inference_server")
            if progress_queue is not None:
                progress_queue.put({
                    "type": "worker_error",
                    "worker": worker_id,
                    "game_local": -1,
                    "message": msg,
                })
            raise RuntimeError(f"Worker {worker_id}: inference_server error -> {msg}")

        return resp

    def predict_batch(states_list):
        if not states_list:
            return []

        request_queue.put((worker_id, True, states_list))
        timeout_s = float(getattr(cfg, "predict_timeout_s", 180))
        try:
            resp = response_queue.get(timeout=timeout_s)
        except Empty:
            if progress_queue is not None:
                progress_queue.put({
                    "type": "worker_error",
                    "worker": worker_id,
                    "game_local": -1,
                    "message": f"Timeout esperando inferencia batch ({timeout_s:.0f}s)",
                })
            raise TimeoutError(f"Worker {worker_id}: timeout esperando inferencia batch ({timeout_s:.0f}s)")

        if isinstance(resp, dict) and resp.get("type") == "inference_error":
            msg = resp.get("message", "error desconocido de inference_server")
            if progress_queue is not None:
                progress_queue.put({
                    "type": "worker_error",
                    "worker": worker_id,
                    "game_local": -1,
                    "message": msg,
                })
            raise RuntimeError(f"Worker {worker_id}: inference_server error -> {msg}")

        if not isinstance(resp, list):
            raise RuntimeError(f"Worker {worker_id}: respuesta batch invalida: {type(resp)}")

        return resp

    return run_n_games_with_predict(n, predict, predict_batch, progress_queue, worker_id, cfg)


def run_n_games_process_entry(n, request_queue, response_queue, progress_queue, result_queue, worker_id, cfg):
    """
    Entry-point para worker de proceso.
    Ejecuta run_n_games y envia resultado/errores por result_queue.
    """
    try:
        # Evitar oversubscription dentro de cada proceso worker
        try:
            torch.set_num_threads(1)
        except Exception:
            pass

        result = run_n_games(n, request_queue, response_queue, progress_queue, worker_id, cfg)
        result_queue.put({
            "type": "worker_done",
            "worker": worker_id,
            "result": result,
        })
    except Exception as exc:
        msg = f"{type(exc).__name__}: {exc}"
        if progress_queue is not None:
            progress_queue.put({
                "type": "worker_error",
                "worker": worker_id,
                "game_local": -1,
                "message": msg,
            })
        try:
            result_queue.put({
                "type": "worker_fail",
                "worker": worker_id,
                "message": msg,
            })
        except Exception:
            pass
        raise


print("Self-play implementado.")

In [ ]:
# ============================================================
# Cell 7: Replay Buffer y Training
# ============================================================

class ReplayBuffer:
    """Buffer circular para almacenar ejemplos de self-play."""
    def __init__(self, max_size):
        self.buffer = deque(maxlen=max_size)

    def add(self, examples):
        """Agregar lista de (state, policy, value) al buffer."""
        self.buffer.extend(examples)

    def sample(self, batch_size):
        """Samplear un batch aleatorio."""
        n = min(batch_size, len(self.buffer))
        indices = np.random.choice(len(self.buffer), size=n, replace=False)
        batch = [self.buffer[i] for i in indices]
        states = np.array([b[0] for b in batch])
        policies = np.array([b[1] for b in batch])
        values = np.array([b[2] for b in batch], dtype=np.float32)
        return states, policies, values

    def __len__(self):
        return len(self.buffer)

    def save(self, path):
        with open(path, 'wb') as f:
            pickle.dump(list(self.buffer), f)

    def load(self, path):
        if os.path.exists(path):
            with open(path, 'rb') as f:
                data = pickle.load(f)
            self.buffer = deque(data, maxlen=self.buffer.maxlen)
            return True
        return False


def train_epoch(net, optimizer, buffer, cfg, device, scaler=None):
    """
    Entrenar una epoca sobre el replay buffer.
    Loss = (z - v)^2 - pi * log(p)
    (weight_decay ya esta en el optimizer como L2)
    """
    net.train()
    total_loss = 0.0
    total_p_loss = 0.0
    total_v_loss = 0.0
    n_batches = 0

    num_batches = max(1, len(buffer) // cfg.batch_size)
    use_amp = bool(getattr(cfg, "use_amp", True) and device.type == "cuda")

    for _ in range(num_batches):
        states, policies, values = buffer.sample(cfg.batch_size)

        states_t = torch.from_numpy(states)
        policies_t = torch.from_numpy(policies)
        values_t = torch.from_numpy(values).unsqueeze(1)

        if device.type == "cuda":
            states_t = states_t.pin_memory().to(device, non_blocking=True)
            policies_t = policies_t.pin_memory().to(device, non_blocking=True)
            values_t = values_t.pin_memory().to(device, non_blocking=True)
        else:
            states_t = states_t.to(device)
            policies_t = policies_t.to(device)
            values_t = values_t.to(device)

        with torch.autocast(device_type="cuda", dtype=torch.float16, enabled=use_amp):
            pred_p, pred_v = net(states_t)

            # Value loss: MSE
            v_loss = F.mse_loss(pred_v, values_t)

            # Policy loss: cross-entropy (pi * log(p))
            p_loss = -torch.mean(torch.sum(policies_t * torch.log(pred_p + 1e-8), dim=1))

            loss = v_loss + p_loss

        optimizer.zero_grad(set_to_none=True)
        if use_amp and scaler is not None:
            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()
        else:
            loss.backward()
            optimizer.step()

        total_loss += loss.item()
        total_p_loss += p_loss.item()
        total_v_loss += v_loss.item()
        n_batches += 1

    if n_batches == 0:
        return 0, 0, 0
    return total_loss / n_batches, total_p_loss / n_batches, total_v_loss / n_batches

print("ReplayBuffer y train_epoch implementados.")

In [ ]:
# ============================================================
# Cell 8: Evaluacion
# ============================================================

def evaluate_mcts(net, cfg, device, n_games=None):
    """Evaluar modelo usando MCTS (greedy, temp=0) y sin Dirichlet noise."""
    if n_games is None:
        n_games = cfg.eval_games
    net.eval()

    wins = 0
    total_score = 0
    scores = []

    eval_cfg = dataclasses.replace(cfg, dir_eps=0.0, temp_init=0.0, temp_final=0.0)

    for _ in tqdm(range(n_games), desc='Eval MCTS', leave=False):
        env = SnakeEnv(eval_cfg.board_size, eval_cfg.max_steps)
        mcts_eval = MCTS(make_predict_fn(net, device), eval_cfg)
        env.reset()

        while not env.done:
            pi = mcts_eval.search(env, temperature=0)
            action = int(np.argmax(pi))
            env.step(action)

        if env.is_win():
            wins += 1
        total_score += env.score
        scores.append(env.score)

    wr = wins / max(n_games, 1)
    avg = total_score / max(n_games, 1)
    return wr, avg, scores


print('Funcion evaluate_mcts implementada (paper-faithful).')


## Loop Principal de Entrenamiento

Secuencia: self-play -> train -> eval MCTS -> champion update -> checkpoint.


In [ ]:
# ============================================================
# Cell 9: Loop Principal de Entrenamiento
# ============================================================

def save_checkpoint(net, best_net, optimizer, iteration, best_win_rate, cfg, buffer):
    """Guardar checkpoint completo en save_dir."""
    ckpt = {
        'iteration': iteration,
        'net': net.state_dict(),
        'best_net': best_net.state_dict(),
        'optimizer': optimizer.state_dict(),
        'best_win_rate': best_win_rate,
        'config': dataclasses.asdict(cfg),
    }
    path = os.path.join(cfg.save_dir, 'latest_checkpoint.pt')
    torch.save(ckpt, path)

    # Guardar mejor modelo por separado
    best_path = os.path.join(cfg.save_dir, 'best_model.pt')
    torch.save(best_net.state_dict(), best_path)

    # Guardar buffer
    buffer_path = os.path.join(cfg.save_dir, 'buffer.pkl')
    buffer.save(buffer_path)

    print(f"  Checkpoint guardado (iter {iteration})")


def load_checkpoint(net, best_net, optimizer, cfg, buffer):
    """Cargar checkpoint desde save_dir si existe."""
    path = os.path.join(cfg.save_dir, 'latest_checkpoint.pt')
    if not os.path.exists(path):
        return 0, 0.0

    ckpt = torch.load(path, map_location=device)
    net.load_state_dict(ckpt['net'])
    best_net.load_state_dict(ckpt['best_net'])
    optimizer.load_state_dict(ckpt['optimizer'])

    buffer_path = os.path.join(cfg.save_dir, 'buffer.pkl')
    buffer.load(buffer_path)

    iteration = ckpt.get('iteration', 0)
    best_wr = ckpt.get('best_win_rate', 0.0)
    print(f"Checkpoint cargado: iteracion {iteration}, best_win_rate={best_wr:.3f}")
    print(f"Buffer restaurado: {len(buffer)} ejemplos")
    return iteration, best_wr


def train_alphasnake(cfg):
    """Loop principal de entrenamiento AlphaSnake."""
    print(f"{'='*60}")
    print(f" AlphaSnake Training")
    print(f" Board: {cfg.board_size}x{cfg.board_size}")
    print(f" Simulations: {cfg.num_simulations}")
    print(f" Games/iter: {cfg.games_per_iter}")
    print(f"{'='*60}\n")

    # Crear redes
    net = AlphaSnakeNet(4, cfg.net_blocks, cfg.net_channels, cfg.board_size).to(device)
    best_net = AlphaSnakeNet(4, cfg.net_blocks, cfg.net_channels, cfg.board_size).to(device)
    best_net.load_state_dict(net.state_dict())

    optimizer = torch.optim.Adam(net.parameters(), lr=cfg.lr, weight_decay=cfg.weight_decay)
    buffer = ReplayBuffer(cfg.buffer_size)
    scaler = torch.amp.GradScaler(device="cuda", enabled=bool(getattr(cfg, "use_amp", True) and device.type == "cuda"))

    # Intentar cargar checkpoint
    start_iter, best_win_rate = load_checkpoint(net, best_net, optimizer, cfg, buffer)

    training_start = time.time()
    for iteration in range(start_iter, cfg.max_iterations):
        iter_start = time.time()
        print(f"\n{'='*60}")
        print(f"  ITERACION {iteration + 1} / {cfg.max_iterations}")
        print(f"{'='*60}")

        # ------------------------------------------------
        # 1. SELF-PLAY PARALELO (procesos + inference batching)
        # ------------------------------------------------
        start_methods = multiprocessing.get_all_start_methods()
        mp_method = "fork" if "fork" in start_methods else "spawn"
        mp_ctx = multiprocessing.get_context(mp_method)

        cpu_total = os.cpu_count() or 8
        requested_workers = int(getattr(cfg, "selfplay_workers", 10))
        oversub_factor = float(getattr(cfg, "worker_oversub_factor", 3.0))
        max_allowed_workers = max(1, int(max(1, cpu_total) * max(1.0, oversub_factor)))
        n_workers = max(1, min(requested_workers, max_allowed_workers))
        best_net.eval()
        all_examples = []
        sp_wins = 0
        sp_scores = []
        sp_moves = []
        sp_lengths = []

        mp_request_queue = mp_ctx.Queue(maxsize=8192)
        response_queues = [mp_ctx.Queue(maxsize=2048) for _ in range(n_workers)]
        progress_queue = mp_ctx.Queue(maxsize=8192)
        result_queue = mp_ctx.Queue(maxsize=max(16, n_workers * 4))

        infer_stats = {"requests": 0, "states": 0, "batches": 0}

        def inference_server():
            batch_timeout = max(0.001, float(getattr(cfg, "inference_timeout_ms", 20)) / 1000.0)
            max_batch = max(8, int(getattr(cfg, "inference_batch_size", 64)))
            use_amp = bool(getattr(cfg, "use_amp", True) and device.type == "cuda")
            try:
                while True:
                    requests = []
                    deadline = time.time() + batch_timeout
                    while len(requests) < max_batch:
                        try:
                            item = mp_request_queue.get(timeout=max(0.001, deadline - time.time()))
                            if item is None:
                                return
                            requests.append(item)
                        except Empty:
                            break
                    if not requests:
                        continue

                    parsed = []
                    flat_states = []

                    for item in requests:
                        if not isinstance(item, tuple):
                            continue

                        if len(item) == 2:
                            wid, payload = item
                            is_batch = False
                        elif len(item) >= 3:
                            wid, is_batch, payload = item[0], bool(item[1]), item[2]
                        else:
                            continue

                        if is_batch:
                            states_list = payload if isinstance(payload, list) else list(payload)
                            start_idx = len(flat_states)
                            flat_states.extend(states_list)
                            parsed.append((int(wid), True, start_idx, len(states_list)))
                        else:
                            start_idx = len(flat_states)
                            flat_states.append(payload)
                            parsed.append((int(wid), False, start_idx, 1))

                    if not parsed or not flat_states:
                        continue

                    infer_stats["requests"] += len(parsed)
                    infer_stats["states"] += len(flat_states)
                    infer_stats["batches"] += 1

                    arr = np.stack(flat_states)
                    t = torch.from_numpy(arr)
                    if device.type == "cuda":
                        t = t.pin_memory().to(device, non_blocking=True)
                    else:
                        t = t.to(device)

                    with torch.no_grad():
                        with torch.autocast(device_type="cuda", dtype=torch.float16, enabled=use_amp):
                            policies, values = best_net(t)

                    p_np = policies.cpu().numpy()
                    v_np = values.cpu().numpy()

                    for wid, is_batch, start_idx, count in parsed:
                        if is_batch:
                            out = [(p_np[start_idx + j], float(v_np[start_idx + j, 0])) for j in range(count)]
                            response_queues[wid].put(out)
                        else:
                            response_queues[wid].put((p_np[start_idx], float(v_np[start_idx, 0])))
            except Exception as exc:
                err = f"{type(exc).__name__}: {exc}"
                print(f"      [Inference ERROR] {err}", flush=True)
                try:
                    progress_queue.put({"type": "inference_error", "message": err})
                except Exception:
                    pass
                for q in response_queues:
                    try:
                        q.put({"type": "inference_error", "message": err})
                    except Exception:
                        pass

        server_thread = threading.Thread(target=inference_server, daemon=True)
        server_thread.start()

        iter_stamp = time.strftime("%Y-%m-%d %H:%M:%S")
        print(f"  [Iter {iteration + 1}] Inicio confirmado: {iter_stamp}")
        print(f"  [Self-play] Arranco con {n_workers} workers y {cfg.games_per_iter} juegos objetivo (backend=processes:{mp_method}, cpu_total={cpu_total}, requested={requested_workers}, max_allowed={max_allowed_workers})")

        sp_start = time.time()
        games_per_worker = [cfg.games_per_iter // n_workers + (1 if i < cfg.games_per_iter % n_workers else 0) for i in range(n_workers)]

        workers = []
        for i in range(n_workers):
            wp = mp_ctx.Process(
                target=run_n_games_process_entry,
                args=(games_per_worker[i], mp_request_queue, response_queues[i], progress_queue, result_queue, i, cfg),
                daemon=True,
            )
            wp.start()
            workers.append(wp)

        pending_workers = set(range(n_workers))
        completed = 0
        games_reported = 0
        verbose_games = bool(getattr(cfg, "verbose_game_updates", True))
        log_every = max(1, int(getattr(cfg, "game_log_interval", 1)))
        heartbeat_s = max(10, int(getattr(cfg, "heartbeat_seconds", 20)))
        last_heartbeat = time.time()

        try:
            with tqdm(total=cfg.games_per_iter, desc=f"Self-play iter {iteration+1}", mininterval=0.5, dynamic_ncols=True) as pbar:
                while pending_workers:
                    drained = 0

                    while True:
                        try:
                            msg = progress_queue.get_nowait()
                        except Empty:
                            break

                        drained += 1
                        msg_type = msg.get("type", "unknown") if isinstance(msg, dict) else "unknown"

                        if msg_type == "worker_start":
                            print(f"      [Worker {msg['worker']}] iniciado | juegos_asignados={msg['games']}", flush=True)
                            continue

                        if msg_type == "game_start":
                            print(f"      [Worker {msg['worker']}] inicio juego local={msg['game_local']}", flush=True)
                            continue

                        if msg_type == "game_tick":
                            if verbose_games:
                                print(
                                    f"      [Tick] worker={msg['worker']} juego={msg['game_local']} "
                                    f"moves={msg['moves']} score={msg['score']} len={msg['length']}",
                                    flush=True,
                                )
                            continue

                        if msg_type == "worker_error":
                            print(
                                f"      [Worker ERROR] worker={msg.get('worker')} juego={msg.get('game_local')} msg={msg.get('message')}",
                                flush=True,
                            )
                            continue

                        if msg_type == "inference_error":
                            print(f"      [Inference ERROR] {msg.get('message')}", flush=True)
                            continue

                        if msg_type == "game_end":
                            games_reported += 1
                            pbar.update(1)
                            if verbose_games and (games_reported == 1 or games_reported % log_every == 0):
                                result_tag = "WIN" if msg["won"] else "LOSE"
                                print(
                                    f"      [Juego {games_reported}/{cfg.games_per_iter}] "
                                    f"worker={msg['worker']} local={msg['game_local']} {result_tag} "
                                    f"score={msg['score']} moves={msg['moves']} len={msg['length']} ex={msg['examples']}",
                                    flush=True,
                                )
                            continue

                    while True:
                        try:
                            rmsg = result_queue.get_nowait()
                        except Empty:
                            break

                        drained += 1
                        rtype = rmsg.get("type") if isinstance(rmsg, dict) else None

                        if rtype == "worker_done":
                            wid = int(rmsg["worker"])
                            ex, w, sc, mv, ln = rmsg["result"]
                            all_examples.extend(ex)
                            sp_wins += w
                            sp_scores.extend(sc)
                            sp_moves.extend(mv)
                            sp_lengths.extend(ln)
                            if wid in pending_workers:
                                pending_workers.remove(wid)
                                completed += 1
                            games_done = len(sp_scores)
                            wr_so_far = 100 * sp_wins / games_done if games_done else 0
                            avg_sc = np.mean(sp_scores) if sp_scores else 0
                            print(f"      [Worker completado] {completed}/{n_workers} | juegos agregados={len(sc)} | acumulado={games_done}/{cfg.games_per_iter} | wins={sp_wins} ({wr_so_far:.1f}%) | avg_score={avg_sc:.1f} | ejemplos={len(all_examples)}", flush=True)
                            continue

                        if rtype == "worker_fail":
                            raise RuntimeError(f"Worker {rmsg.get('worker')} failed: {rmsg.get('message')}")

                    if pending_workers:
                        now = time.time()
                        if drained == 0 and now - last_heartbeat >= heartbeat_s:
                            reqs = infer_stats["requests"]
                            states_n = infer_stats["states"]
                            batches = infer_stats["batches"]
                            avg_states = states_n / max(batches, 1)
                            print(
                                f"      [Heartbeat] juegos={games_reported}/{cfg.games_per_iter} | workers={completed}/{n_workers} | infer_reqs={reqs} | infer_states={states_n} | infer_batches={batches} | avg_states_batch={avg_states:.1f}",
                                flush=True,
                            )
                            last_heartbeat = now
                        time.sleep(0.2)
        finally:
            for wp in workers:
                if wp.is_alive():
                    wp.join(timeout=0.2)
            # cerrar inference server
            mp_request_queue.put(None)
            server_thread.join(timeout=10)

            # si quedaron workers vivos, forzar cierre
            for wp in workers:
                if wp.is_alive():
                    wp.terminate()
                    wp.join(timeout=2)

        reqs = infer_stats["requests"]
        states_n = infer_stats["states"]
        batches = infer_stats["batches"]
        avg_states = states_n / max(batches, 1)
        print(f"    Inference batching -> reqs={reqs}, states={states_n}, batches={batches}, avg_states_batch={avg_states:.1f}")

        sp_time = time.time() - sp_start
        sp_wr = sp_wins / cfg.games_per_iter
        sp_scores_arr = np.array(sp_scores)
        sp_moves_arr = np.array(sp_moves)
        sp_lengths_arr = np.array(sp_lengths)

        ejemplos_per_min = len(all_examples) / (sp_time / 60.0) if sp_time > 0 else 0
        print(f"  --- Self-play ({sp_time:.0f}s, {n_workers} procesos) ---")
        print(f"    Juegos: {cfg.games_per_iter} | Wins: {sp_wins} ({100*sp_wr:.1f}%) | Ejemplos: {len(all_examples)} ({ejemplos_per_min:.0f}/min)")
        print(f"    Score  -> min: {sp_scores_arr.min():.0f}  med: {np.median(sp_scores_arr):.0f}  max: {sp_scores_arr.max():.0f}  avg: {sp_scores_arr.mean():.1f}")
        print(f"    Length -> min: {sp_lengths_arr.min():.0f}  med: {np.median(sp_lengths_arr):.0f}  max: {sp_lengths_arr.max():.0f}  (max: {cfg.board_size**2})")
        print(f"    Moves  -> min: {sp_moves_arr.min():.0f}  med: {np.median(sp_moves_arr):.0f}  max: {sp_moves_arr.max():.0f}")

        buffer.add(all_examples)
        print(f"    Buffer total: {len(buffer)}")

        # ------------------------------------------------
        # 2. TRAINING
        # ------------------------------------------------
        train_start = time.time()
        last_loss, last_p, last_v = 0.0, 0.0, 0.0
        for epoch in tqdm(range(cfg.epochs_per_iter), desc=f"Training iter {iteration+1}", leave=False):
            last_loss, last_p, last_v = train_epoch(net, optimizer, buffer, cfg, device, scaler=scaler)
            if (epoch + 1) % 5 == 0 or epoch == 0:
                tqdm.write(f"    Epoch {epoch+1}/{cfg.epochs_per_iter}: loss={last_loss:.4f} (policy={last_p:.4f}, value={last_v:.4f})")
        train_time = time.time() - train_start
        print(f"  --- Training ({train_time:.0f}s, {cfg.epochs_per_iter} epochs) ---")
        print(f"    Loss: total={last_loss:.4f} | policy={last_p:.4f} | value={last_v:.4f}")

        # ------------------------------------------------
        # 3. EVALUACION MCTS (paper-faithful)
        # ------------------------------------------------
        eval_start = time.time()
        wr_new_mcts, avg_new_mcts, _ = evaluate_mcts(net, cfg, device, n_games=cfg.eval_games)
        eval_time = time.time() - eval_start

        print(f"  --- Eval MCTS ({cfg.eval_games} juegos, {eval_time:.0f}s) ---")
        print(f"    Nuevo: win_rate={100*wr_new_mcts:.1f}% | avg_score={avg_new_mcts:.1f}")
        print(f"    Umbral aceptacion: {100*cfg.accept_threshold:.1f}% | Best actual: {100*best_win_rate:.1f}%")

        # ------------------------------------------------
        # 4. CHAMPION UPDATE (paper-faithful)
        # ------------------------------------------------
        required_wr = max(cfg.accept_threshold, best_win_rate)
        if wr_new_mcts >= required_wr:
            print('  >>> NUEVO CHAMPION aceptado <<<')
            best_net.load_state_dict(net.state_dict())
            best_win_rate = wr_new_mcts
        else:
            print('  Champion previo retenido.')
            net.load_state_dict(best_net.state_dict())

        # ------------------------------------------------
        # 5. RESUMEN ITERACION
        # ------------------------------------------------
        iter_time = time.time() - iter_start
        total_elapsed = time.time() - training_start
        iters_done = iteration - start_iter + 1
        iters_left = cfg.max_iterations - iteration - 1
        eta_sec = (iter_time * iters_left) if iters_left > 0 else 0
        eta_min = eta_sec / 60

        print(f"  --- Resumen iter {iteration+1}/{cfg.max_iterations} ---")
        print(f"    Tiempos: self-play={sp_time:.0f}s  train={train_time:.0f}s  eval={eval_time:.0f}s  total={iter_time:.0f}s")
        print(f"    Acumulado: {total_elapsed/60:.1f} min | Restantes: {iters_left} iters | ETA ~{eta_min:.0f} min")
        print(f"    Buffer: {len(buffer)} posiciones | Mejor win_rate (MCTS): {best_win_rate:.3f}")
        if (iteration + 1) % cfg.checkpoint_interval == 0:
            save_checkpoint(net, best_net, optimizer, iteration + 1, best_win_rate, cfg, buffer)
            print(f"    Checkpoint guardado (iter {iteration+1})")

    print(f"\n{'='*60}")
    print(f"  ENTRENAMIENTO COMPLETADO")
    print(f"  Mejor win rate (policy): {best_win_rate:.3f}")
    print(f"{'='*60}")

    return best_net

print("Loop de entrenamiento definido. Ejecutar la siguiente celda para entrenar.")


### Takeaway - Champion
La seleccion de champion usa evaluacion MCTS real y umbral `accept_threshold=0.55`.
El modelo solo se promueve si supera el mejor win-rate acumulado y ese umbral.


In [ ]:
# ============================================================
# Cell X: Benchmark rapido de throughput (opcional)
# ============================================================
# Ejecuta esta celda solo si quieres estimar el tiempo por iteracion.

net_bench = AlphaSnakeNet(4, config.net_blocks, config.net_channels, config.board_size).to(device)
opt_bench = torch.optim.Adam(net_bench.parameters(), lr=config.lr, weight_decay=config.weight_decay)
buf_bench = ReplayBuffer(max(config.batch_size * 8, 2048))

synth_states = np.random.randn(config.batch_size, 4, config.board_size, config.board_size).astype(np.float32)
synth_policies = np.random.dirichlet(alpha=np.ones(4), size=config.batch_size).astype(np.float32)
synth_values = np.random.uniform(-1, 1, size=(config.batch_size,)).astype(np.float32)
for _ in range(max(8, 2048 // config.batch_size)):
    buf_bench.add(list(zip(synth_states, synth_policies, synth_values)))

bench_start = time.time()
_ = train_epoch(net_bench, opt_bench, buf_bench, config, device)
bench_elapsed = time.time() - bench_start
print(f'Benchmark train_epoch: {bench_elapsed:.3f}s (batch_size={config.batch_size})')


In [ ]:
# ============================================================
# Cell 10: Ejecutar entrenamiento
# ============================================================
# Esta celda lanza el loop completo.
# Puedes interrumpir y reanudar: usa checkpoints en config.save_dir.

best_model = train_alphasnake(config)


## Evaluacion Final + Export ONNX

Se valida el champion con MCTS real y luego se exporta a ONNX con
el contrato requerido por `ai-bot.js` y `ai-mcts-worker.js`.


In [ ]:
# ============================================================
# Cell 11: Evaluacion final con MCTS
# ============================================================

best_model_path = os.path.join(config.save_dir, 'best_model.pt')
if os.path.exists(best_model_path):
    best_model = AlphaSnakeNet(4, config.net_blocks, config.net_channels, config.board_size).to(device)
    best_model.load_state_dict(torch.load(best_model_path, map_location=device))
    print('Mejor modelo cargado desde save_dir.')

print()
print('Evaluacion final con MCTS (puede tardar)...')
final_games = max(100, config.eval_games)
wr, avg_score, scores = evaluate_mcts(best_model, config, device, n_games=final_games)

print()
print(f'Resultados finales ({final_games} juegos MCTS):')
print(f'  Win rate: {wr:.3f} ({int(wr*final_games)}/{final_games})')
print(f'  Avg score: {avg_score:.1f} / {config.board_size**2 - 3}')
print(f'  Min score: {min(scores)}, Max score: {max(scores)}')
print(f'  Median score: {np.median(scores):.0f}')


In [ ]:
# ============================================================
# Cell 12: Exportar y validar ONNX
# ============================================================
import onnx
import onnxruntime as ort

best_model.eval()
best_model.cpu()

dummy_input = torch.randn(1, 4, config.board_size, config.board_size)
onnx_path = os.path.join(config.save_dir, 'alphasnake.onnx')

torch.onnx.export(
    best_model,
    dummy_input,
    onnx_path,
    export_params=True,
    opset_version=13,
    do_constant_folding=True,
    input_names=['state'],
    output_names=['policy', 'value'],
    dynamic_axes={
        'state': {0: 'batch_size'},
        'policy': {0: 'batch_size'},
        'value': {0: 'batch_size'},
    }
)

model_onnx = onnx.load(onnx_path)
onnx.checker.check_model(model_onnx)
print(f'Modelo ONNX exportado y verificado: {onnx_path}')

ort_session = ort.InferenceSession(onnx_path)
test_input = np.random.randn(1, 4, config.board_size, config.board_size).astype(np.float32)

with torch.no_grad():
    pt_policy, pt_value = best_model(torch.from_numpy(test_input))
pt_policy = pt_policy.numpy()
pt_value = pt_value.numpy()

ort_policy, ort_value = ort_session.run(None, {'state': test_input})

print()
print('Verificacion PyTorch vs ONNX:')
print(f'  Policy diff max: {np.abs(pt_policy - ort_policy).max():.8f}')
print(f'  Value diff max:  {np.abs(pt_value - ort_value).max():.8f}')
print(f'  Tamano modelo: {os.path.getsize(onnx_path) / (1024 * 1024):.2f} MB')


### Takeaway - ONNX
Se valida el contrato exacto para el juego:
- Input `state`: `[batch, 4, 10, 10]`
- Outputs: `policy` `[batch, 4]`, `value` `[batch, 1]`


In [ ]:
# ============================================================
# Cell 13: Integracion con el juego existente
# ============================================================

REPO_MODEL_PATH = '/Users/JuanCamiloTorresUrrego/Documents/tiktok/arcade/games/snake/ai/alphasnake.onnx'

print('Modelo ONNX generado en:')
print(f'  {onnx_path}')
print()
print('Pasos de integracion:')
print(f'1) Copiar el modelo: cp "{onnx_path}" "{REPO_MODEL_PATH}"')
print('2) Levantar juego: npm run start:game')
print('3) Abrir: http://localhost:3000/snake/ai.html')
print('4) En la UI, usar modo MCTS y slider en 400 simulaciones para condiciones paper.')


## Takeaways finales

- Notebook listo para Vast.ai con perfiles `paper_strict` y `smoke`.
- Pipeline completo reproducible: self-play -> train -> eval MCTS -> champion -> export ONNX.
- Integracion con el juego lista sin adaptar el modelo al engine.
